# Use the Saved model to make predictions for new data

## Step 1 : Import all Dependencies
### Import Pyspark and create a SparkSession, set the Sparkcontext.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession, functions, types
import random
spark = SparkSession\
    .builder\
    .appName("Transform&Load")\
    .config("spark.driver.extraClassPath","/home/jim/spark-2.4.0-bin-hadoop2.7/jars/mysql-connector-java-5.1.49.jar")\
    .getOrCreate()
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

### Create an H2O cluster inside the Spark cluster

In [3]:
from pysparkling import *
hc = H2OContext.getOrCreate(spark)
import h2o

Method getOrCreate with spark argument is deprecated. Please use either just getOrCreate() or if you need to pass extra H2OConf, use getOrCreate(conf). The spark argument will be removed in release 3.32.


Connecting to H2O server at http://172.29.157.231:54321 ... successful.


H2O_cluster_uptime:,12 secs
H2O_cluster_timezone:,America/Vancouver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,sparkling-water-jim_local-1592717773483
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,878 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.30.0.4-1-2.4
 * H2O name: sparkling-water-jim_local-1592717773483
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,172.29.157.231,54321)
  ------------------------

  Open H2O Flow in browser: http://172.29.157.231:54324 (CMD + click in Mac OSX)

    


### Resuse the Saved Random Forest Model

In [4]:
model = h2o.load_model('/home/jim/Documents/Data_Analytics_Projects/Insurance_Customer _Churn/Misc/models/RF_Insurance_model/DRF_model_python_1591600273347_1')
model

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1591600273347_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,375810.0,18.0,20.0,19.86,538.0,629.0,594.06




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.2639644080003918
RMSE: 0.5137746665615109
MAE: 0.49683306728584103
RMSLE: 0.3594929987528265
Mean Residual Deviance: 0.2639644080003918

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.2540269320286576
RMSE: 0.5040108451498416
MAE: 0.4963127403375496
RMSLE: 0.35363843779933546
Mean Residual Deviance: 0.2540269320286576

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,mae,0.4963061,0.0044579403,0.49884567,0.49887803,0.48952442,0.4946703,0.4945817,0.4956139,0.5041945,0.4912182,0.49450418,0.5010299
1,mean_residual_deviance,0.2540221,0.0044468003,0.25660747,0.25628483,0.24759479,0.25241446,0.25190705,0.25307295,0.26184928,0.24892348,0.25236067,0.2592059
2,mse,0.2540221,0.0044468003,0.25660747,0.25628483,0.24759479,0.25241446,0.25190705,0.25307295,0.26184928,0.24892348,0.25236067,0.2592059
3,r2,-0.020484393,0.017332442,-0.026490971,-0.03650415,2.5801553E-4,-0.009878392,-0.009735626,-0.030263606,-0.047829106,0.002726303,-0.01030276,-0.036823645
4,residual_deviance,0.2540221,0.0044468003,0.25660747,0.25628483,0.24759479,0.25241446,0.25190705,0.25307295,0.26184928,0.24892348,0.25236067,0.2592059
5,rmse,0.5039887,0.004406204,0.5065644,0.5062458,0.49758896,0.5024087,0.5019034,0.50306356,0.51171213,0.49892232,0.5023551,0.50912267
6,rmsle,0.35355663,0.0060714753,0.35622177,0.36238262,0.34413302,0.352373,0.3567748,0.34493846,0.35786304,0.34783491,0.35528642,0.3577583



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2020-06-08 00:44:03,12.717 sec,0.0,NaN,NaN,NaN
1,,2020-06-08 00:44:03,12.777 sec,1.0,0.672536,0.476249,0.452305
2,,2020-06-08 00:44:03,12.811 sec,2.0,0.670496,0.489213,0.449564
3,,2020-06-08 00:44:03,12.839 sec,3.0,0.654237,0.488699,0.428026
4,,2020-06-08 00:44:03,12.860 sec,4.0,0.636324,0.485949,0.404908
5,,2020-06-08 00:44:03,12.881 sec,5.0,0.629090,0.495190,0.395754
6,,2020-06-08 00:44:03,12.901 sec,6.0,0.611925,0.493218,0.374452
7,,2020-06-08 00:44:03,12.921 sec,7.0,0.601446,0.495600,0.361738
8,,2020-06-08 00:44:03,12.942 sec,8.0,0.592244,0.495808,0.350753
9,,2020-06-08 00:44:03,12.965 sec,9.0,0.583014,0.496234,0.339905



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Total Written Premium,1472.453613,1.000000,0.048682
1,Total Claim Amount Index,1440.992920,0.978634,0.047642
2,Losses,1439.602417,0.977689,0.047596
3,Commissions,1420.929321,0.965008,0.046978
4,Monthly Premium Auto,1414.790771,0.960839,0.046775
5,Income,1414.226562,0.960456,0.046757
6,Customer Lifetime Value,1369.011841,0.929749,0.045262
7,Growth Rate,1358.496460,0.922607,0.044914
8,Company Index,1357.218872,0.921740,0.044872
9,Loss Ratio,1351.888062,0.918119,0.044696



See the whole table with table.as_data_frame()


### Read just one row from the Database table and convert it to a Spark Dataframe

In [44]:
spark_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/Insurance")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "Insurance_numeric").option("user", "jsully")\
    .option("password", "whatisreal1").load()
spark_df = spark_df.limit(1)
spark_df.show(5)

+-----------------------+---+-----------------+------+--------------------+---------------------+------+----------+-----------+-----------+-----------------------+-----------------------------+-------------------------+------------------+---------------------------+---------+--------------+--------------+---------------+-----------------------+------------+-------------------+--------------------+-----------------+-------------------+----------------------+-------------------+------------------------+--------------+---------+-------------+--------------------------+-----+
|Customer Lifetime Value|Age|Effective To Date|Income|Monthly Premium Auto|Total Written Premium|Losses|Loss Ratio|Growth Rate|Commissions|Months Since Last Claim|Months Since Policy Inception|Number of Open Complaints|Number of Policies|Number of previous policies|CityIndex|Response Index|Coverage Index|Education Index|Employment_Status Index|Gender Index|Location_Code Index|Marital Status Index|Policy_Type Index|Pol

#### Perform some preprocessing

In [45]:
spark_df = spark_df.withColumn("Effective To Date",functions.split("Effective To Date", "-").getItem(2))
spark_df = spark_df.withColumn("Effective To Date", spark_df["Effective To Date"].cast(types.IntegerType()))
spark_df = spark_df.drop('Churn')

### Convert it to H2O dataframe, feed it to the model and collect the result as a prediction

In [46]:
predictions = model.predict(hc.asH2OFrame(spark_df))
df = hc.asSparkFrame(predictions)
df = df.withColumn("predict", functions.round("predict", 0))
df = df.withColumn("predict", df["predict"].cast(types.IntegerType()))
df.show()

drf prediction progress: |████████████████████████████████████████████████| 100%
+-------+
|predict|
+-------+
|      1|
+-------+



## Now we will use our own values to come up with a prediction for "Churn"

### Initiate dictionary with random variables

In [5]:
customer = {
    'Customer ID':'CS14069',
    'City':'Mumbai',
    'Customer Lifetime Value':6077.675806,
    'Age':40,
    'Response':'No',
    'Coverage':'Extended',
    'Education':'Diploma',
    'Effective To Date':'05-03-2019',
    'Employment_Status':'Unemployed',
    'Gender':'Female',
    'Income':40000,
    'Location_Code':'Suburban',
    'Marital Status':'Married',
    'Monthly Premium Auto':110,
    'Total Written Premium':8000,
    'Losses':9000,
    'Loss Ratio':0.429590,
    'Growth Rate':1.10,
    'Commissions':5000,
    'Months Since Last Claim':13,
    'Months Since Policy Inception':20,
    'Number of Open Complaints':2,
    'Number of Policies':2,
    'Number of previous policies':2,
    'Policy_Type':'Corporate',
    'Policy_Rating':'Average',
    'Renew_Offer_Type':'Offer 2',
    'Sales_Channel':'Other',
    'Total Claim Amount':60000.0,
    'Feedback':'Average',
    'Name':'Charles Jameson',
    'Address':'8888, Whitefield Garden',
    'Phone_no':'792.000.7028',
    'Job':'Engineer',
    'Company':'Infosys',
    'Credit Card Provider':'VISA 16 digit',
    'Email':'charles@yahoo.com',
    'SSN':'359-15-7355'
}

### Convert it to a Spark Dataframe

In [6]:
df = sc.parallelize([customer]).toDF()
df.show()

Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead


+--------------------+---+------+-----------+-------+--------+--------------------+-----------+-----------------------+---------+-----------------+-----------------+-----------------+--------+------+-----------+------+--------+-------------+----------+------+--------------+--------------------+-----------------------+-----------------------------+---------------+-------------------------+------------------+---------------------------+------------+-------------+-----------+----------------+--------+-----------+-------------+------------------+---------------------+
|             Address|Age|  City|Commissions|Company|Coverage|Credit Card Provider|Customer ID|Customer Lifetime Value|Education|Effective To Date|            Email|Employment_Status|Feedback|Gender|Growth Rate|Income|     Job|Location_Code|Loss Ratio|Losses|Marital Status|Monthly Premium Auto|Months Since Last Claim|Months Since Policy Inception|           Name|Number of Open Complaints|Number of Policies|Number of previous 

### Transform it to a format that will be accepted by the model

In [7]:
cols_to_drop = ["Customer ID","Name","Address","Phone_no","Email","SSN"]
df = df.drop(*cols_to_drop)
df = df.withColumn("Customer Lifetime Value", functions.round("Customer Lifetime Value", 2))
df = df.withColumn("Loss Ratio", functions.round("Loss Ratio", 3))
df = df.withColumn("Growth Rate", functions.round("Growth Rate", 3))
df = df.withColumn("Total Claim Amount", functions.round("Total Claim Amount", 3))
df = df.withColumn("Job",functions.split("Job", ",").getItem(0))
df = df.withColumn("Company",functions.reverse(functions.split("Company", ",")).getItem(0))
df.show()

+---+------+-----------+-------+--------+--------------------+-----------------------+---------+-----------------+-----------------+--------+------+-----------+------+--------+-------------+----------+------+--------------+--------------------+-----------------------+-----------------------------+-------------------------+------------------+---------------------------+-------------+-----------+----------------+--------+-------------+------------------+---------------------+
|Age|  City|Commissions|Company|Coverage|Credit Card Provider|Customer Lifetime Value|Education|Effective To Date|Employment_Status|Feedback|Gender|Growth Rate|Income|     Job|Location_Code|Loss Ratio|Losses|Marital Status|Monthly Premium Auto|Months Since Last Claim|Months Since Policy Inception|Number of Open Complaints|Number of Policies|Number of previous policies|Policy_Rating|Policy_Type|Renew_Offer_Type|Response|Sales_Channel|Total Claim Amount|Total Written Premium|
+---+------+-----------+-------+--------+-

In [8]:
from pyspark.ml.feature import StringIndexer
indexer_list = []
categ_cols = ['City','Response','Coverage','Education','Employment_Status','Gender','Location_Code','Marital Status','Policy_Type','Policy_Rating','Renew_Offer_Type','Sales_Channel','Total Claim Amount','Feedback','Job','Company','Credit Card Provider']
for i in categ_cols:
    indexer_list.append(StringIndexer(inputCol=i, outputCol=i+" Index"))
for j in indexer_list:
    df = j.fit(df).transform(df)
df = df.select([c for c in df.columns if c not in categ_cols])
df = df.withColumn("Effective To Date",functions.split("Effective To Date", "-").getItem(2))
df = df.withColumn("Effective To Date", df["Effective To Date"].cast(types.IntegerType()))
df.show()

+---+-----------+-----------------------+-----------------+-----------+------+----------+------+--------------------+-----------------------+-----------------------------+-------------------------+------------------+---------------------------+---------------------+----------+--------------+--------------+---------------+-----------------------+------------+-------------------+--------------------+-----------------+-------------------+----------------------+-------------------+------------------------+--------------+---------+-------------+--------------------------+
|Age|Commissions|Customer Lifetime Value|Effective to Date|Growth Rate|Income|Loss Ratio|Losses|Monthly Premium Auto|Months Since Last Claim|Months Since Policy Inception|Number of Open Complaints|Number of Policies|Number of previous policies|Total Written Premium|City Index|Response Index|Coverage Index|Education Index|Employment_Status Index|Gender Index|Location_Code Index|Marital Status Index|Policy_Type Index|Policy_

### Obtain a result using the saved model

In [13]:
pred = model.predict(hc.asH2OFrame(df))
prediction_df = hc.asSparkFrame(pred)
prediction_df = prediction_df.withColumn("predict", functions.round("predict", 0))
prediction_df = prediction_df.withColumn("predict", prediction_df["predict"].cast(types.IntegerType()))
prediction_df.show()

drf prediction progress: |████████████████████████████████████████████████| 100%
+-------+
|predict|
+-------+
|      1|
+-------+



In [26]:
result = prediction_df.collect()[0].predict
if result is 0:
    print('This customer will not Churn')
elif result is 1:
    print('This Customer is likely to Churn')

This Customer is likely to Churn
